# Amazon SageMaker Clarifyを用いたデータバイアス検知


## Amazon Science: _[How Clarify helps machine learning developers detect unintended bias](https://www.amazon.science/latest-news/how-clarify-helps-machine-learning-developers-detect-unintended-bias)_ 

[<img src="img/amazon_science_clarify.png"  width="100%" align="left">](https://www.amazon.science/latest-news/how-clarify-helps-machine-learning-developers-detect-unintended-bias)

# 用語集
https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-detect-data-bias.html

* **バイアス**:
年齢や収入層などの異なるグループ間で、学習データやモデルの予測動作に生じる不均衡のこと。バイアスは、モデルの学習に使用したデータやアルゴリズムから生じることがあります。例えば、MLモデルが主に中高年のデータで学習された場合、若年層や高齢者に対する予測を行う際に精度が低くなる可能性があります。

* **バイアスメトリクス**: 
潜在的なバイアスの度合いを示す数値を返す関数。

* **バイアスレポート**:
分析対象のデータセット、またはデータセットとモデルの組み合わせに対するバイアスメトリクスのコレクション。

* **ラベル**:
機械学習モデルのトレーニングのターゲットとなる特徴量。

* **ポジティブラベル値**:
サンプル内の特定の人口集団（年代、性別など）でよく観測されるラベル値。言い換えれば、サンプルがポジティブな結果を持つことを示しています。

* **ネガティブラベル値**:
サンプル内の特定の人口集団（年代、性別など）であまり観測されないラベル値。言い換えれば、サンプルがネガティブな結果を持つことを示しています。

* **ファセット**:
バイアスの分析対象となる属性を含むカラムまたは特徴量のこと。（訳注: 例えばデータセットに男女間の差異がないかを分析したい場合は、「性別」のカラムがファセット（側面）となります。）

* **ファセット値**:
バイアスが含まれ得る属性の特徴値。

# トレーニング前バイアスメトリクス
https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-data-bias.html

* **クラス不均衡、Class Imbalance (CI)**:
異なるファセット値間のサンプル数の不均衡を測定。（訳注: 男女のサンプル数に不均衡がないか、など）

* **正例ラベル比率の差、Difference in Proportions of Labels (DPL)**:
異なるファセット値間でポジティブなラベル比率の不均衡を測定。（訳注: 国籍によってローンの審査の通過率に不均衡がないか、など）

* **カルバック・ライブラー・ダイバージェンス、Kullback-Leibler Divergence (KL)**:
異なるファセットのラベル分布がエントロピー的にどの程度乖離しているかを測定。

* **イェンセン・シャノン・ダイバージェンス、Jensen-Shannon Divergence (JS)**:
異なるファセットのラベル分布がエントロピー的にどの程度乖離しているかを測定。

* **Lpノルム、Lp-norm (LP)**:
データセット内の異なるファセットのラベル分布同士のpノルム差を測定。

* **全変動距離、Total Variation Distance (TVD)**:
データセット内の異なるファセットのラベル分布同士のL1ノルムの半分を測定。

* **コルモゴロフ・スミルノフ、Kolmogorov-Smirnov (KS)**:
データセット内の異なるファセットのラベル分布同士の最大ダイバージェンスを測定。

* **条件付き人口統計差異、Conditional Demographic Disparity (CDD)**:
あるファセットのポジティブとネガティブのラベル比率の差異をサブグループごとに測定。

In [ ]:
import boto3
import sagemaker
import pandas as pd
import numpy as np

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

import botocore.config

config = botocore.config.Config(
    user_agent_extra='dsoaws/1.0'
)

sm = boto3.Session().client(service_name="sagemaker", 
                            region_name=region,
                            config=config)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format='retina'

# データセットの分析

各商品カテゴリーのPandas DataFrameを作成します。

## データセットカラムの説明
- `marketplace`: 二文字の国コード（今回はすべて「US」）。
- `customer_id`: それぞれの書き手のレビュー集約に使われるランダムID。
- `review_id`: レビューのユニークID。
- `product_id`: Amazon標準識別番号（ASIN）。`http://www.amazon.com/dp/<ASIN>` が商品の詳細ページへのリンクとなる。
- `product_parent`: ASINの親。ひとつの親に対して複数のASINが存在しうる（同じ商品の色違いやフォーマット違いなど）。
- `product_title`: 商品のタイトル表記。
- `product_category`: レビューのグループ化に使う大まかな商品カテゴリー（このケースでは「デジタルビデオ」など）
- `star_rating`: レビューの星評価（1〜5）。
- `helpful_votes`: レビューへの「役に立った」投票の個数。
- `total_votes`: レビューへの全投票数。
- `vine`: レビューが[Vine](https://www.amazon.com/gp/vine/help)先取りプログラムの一環で書かれたか否か。
- `verified_purchase`: レビューが検証済みの購入に対するものか否か。
- `review_headline`: レビューそのもののタイトル。
- `review_body`: レビュー本体のテキスト。
- `review_date`: レビューが投稿された日付。
- `year`: レビュー投稿日付から取得した年情報。

時間短縮のため、データセット全体ではなく「ギフトカード」、「デジタルソフトウェア」、「デジタルビデオゲーム」カテゴリーの一部のデータに対して分析を行います。

In [ ]:
import csv

df_giftcards = pd.read_csv(
    "./data-clarify/amazon_reviews_us_Gift_Card_v1_00.tsv.gz",
    delimiter="\t",
    quoting=csv.QUOTE_NONE,
    compression="gzip",
)

df_software = pd.read_csv(
    "./data-clarify/amazon_reviews_us_Digital_Software_v1_00.tsv.gz",
    delimiter="\t",
    quoting=csv.QUOTE_NONE,
    compression="gzip",
)

df_videogames = pd.read_csv(
    "./data-clarify/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz",
    delimiter="\t",
    quoting=csv.QUOTE_NONE,
    compression="gzip",
)

df = pd.concat([df_giftcards, df_software, df_videogames], ignore_index=True, sort=False)
df.head()

データセットにある偏り（バイアス）を確認してみましょう。

In [ ]:
import seaborn as sns

sns.countplot(data=df, x="star_rating", hue="product_category")

### データのアップロード

一般に、SageMaker のジョブの入力となるデータは Amaozn S3 に配置します。S3に置いたデータに対して、後ほどSageMaker Clarifyの処理ジョブを実行します。

In [ ]:
!mkdir -p ./transformed/

path = "./amazon_reviews_us_giftcards_software_videogames.csv"
df.to_csv(path, index=False, header=True)

data_s3_uri = sess.upload_data(bucket=bucket, key_prefix="bias/transformed", path=path)
data_s3_uri

# バイアス分析

### `DataConfig` のセットアップ

`DataConfig` では分析対象のテーブルデータの情報を教えてあげます。
以下のように、Amazon S3上に置いたトレーニングデータのパスや、バイアス分析結果のレポートの保存場所を与えたり、予測対象のラベルはどれで、テーブルデータにはどんなカラムがあるのか、データセットのフォーマットは何なのかということを定義します。

In [ ]:
from sagemaker import clarify

bias_s3_prefix = "bias/generated_bias_report"
bias_report_output_path = "s3://{}/{}/data".format(bucket, bias_s3_prefix)

data_config = clarify.DataConfig(
    s3_data_input_path=data_s3_uri,
    s3_output_path=bias_report_output_path,
    label="star_rating",
    headers=df.columns.to_list(),
    dataset_type="text/csv",
)

### `BiasConfig` のセットアップ

SageMaker Clarifyでは、分析対象のカラム（`facets`）と何が望ましい結果なのか（`label_values_or_threshold`）を教えてあげる必要があります。

これらの情報は `BiasConfig` API で指定します。ここでは `star_rating==5` と `star_rating==4` が望ましい結果となります。`product_category` が今回分析するファセットです。

In [ ]:
bias_config = clarify.BiasConfig(
    label_values_or_threshold=[5, 4], 
    facet_name="product_category"
)

### SageMaker Clarify Processing Jobのセットアップ

In [ ]:
processor = clarify.SageMakerClarifyProcessor(
    role=role, 
    instance_count=1, 
    instance_type="ml.m5.xlarge", 
    sagemaker_session=sess
)

### Processing Jobを実行

トレーニング前バイアスメトリクスを計算するジョブを投げます。

In [ ]:
processor.run_pre_training_bias(
    data_config=data_config, 
    data_bias_config=bias_config, 
    methods=["CI", "DPL", "KL", "JS", "LP", "TVD", "KS"],
    wait=False, 
    logs=False
)

In [ ]:
bias_processing_job_name = processor.latest_job.job_name
print(bias_processing_job_name)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, bias_processing_job_name
        )
    )
)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, bias_processing_job_name
        )
    )
)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}?region={}&prefix={}/">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(
            bucket, region, bias_s3_prefix
        )
    )
)

In [ ]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(
    processing_job_name=bias_processing_job_name, sagemaker_session=sess
)

### _このセルの実行には5〜10分程度かかります。_

In [ ]:
%%time

running_processor.wait(logs=False)

### バイアスレポートを読む

In [ ]:
!aws s3 ls $bias_report_output_path/

In [ ]:
!aws s3 cp --recursive $bias_report_output_path ./generated_bias_report/data/

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML('<b>Review <a target="blank" href="./generated_bias_report/data/report.html">Unbalanced Bias Report</a></b>')
)

# `product_category` と `star_rating` に対してデータセットをバランス化

今回はアンダーサンプリング（レビュー数が一番少ない星評価に合わせる）というテクニックを用いてデータセットをバランス化しましょう。

In [ ]:
df_group_by = df.groupby(["product_category", "star_rating"])
df_balanced_data = df_group_by.apply(lambda x: x.sample(df_group_by.size().min()).reset_index(drop=True))

In [ ]:
import seaborn as sns

sns.countplot(data=df_balanced_data, x="star_rating", hue="product_category")

# SageMaker Clarifyを用いてバランス化したデータセットのバイアスを分析

In [ ]:
path_balanced = "./amazon_reviews_us_giftcards_software_videogames_balanced.csv"
df_balanced_data.to_csv(path_balanced, index=False, header=True)

balanced_data_s3_uri = sess.upload_data(bucket=bucket, key_prefix="bias/data_balanced", path=path_balanced)
balanced_data_s3_uri

### `DataConfig` のセットアップ

In [ ]:
from sagemaker import clarify

bias_s3_prefix = "bias/generated_bias_report"
bias_report_balanced_output_path = "s3://{}/{}/data_balanced".format(bucket, bias_s3_prefix)

balanced_data_config = clarify.DataConfig(
    s3_data_input_path=balanced_data_s3_uri,
    s3_output_path=bias_report_balanced_output_path,
    label="star_rating",
    headers=df_balanced_data.columns.to_list(),
    dataset_type="text/csv",
)

### `BiasConfig` のセットアップ

SageMaker Clarifyでは、分析対象のカラム（`facets`）と何が望ましい結果なのか（`label_values_or_threshold`）を教えてあげる必要があります。

これらの情報は `BiasConfig` API で指定します。ここでは `star_rating==5` と `star_rating==4` が望ましい結果となります。`product_category` が今回分析するファセットです。

In [ ]:
bias_config = clarify.BiasConfig(
    label_values_or_threshold=[5, 4], 
    facet_name="product_category" 
)

### SageMaker Clarify Processing Jobのセットアップ

In [ ]:
processor = clarify.SageMakerClarifyProcessor(
    role=role, 
    instance_count=1, 
    instance_type="ml.m5.xlarge", 
    sagemaker_session=sess
)

In [ ]:
processor.run_pre_training_bias(
    data_config=balanced_data_config, 
    data_bias_config=bias_config, 
    methods=["CI", "DPL", "KL", "JS", "LP", "TVD", "KS"],
    wait=False, 
    logs=False
)

In [ ]:
balanced_bias_processing_job_name = processor.latest_job.job_name
print(balanced_bias_processing_job_name)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, balanced_bias_processing_job_name
        )
    )
)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, balanced_bias_processing_job_name
        )
    )
)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}?region={}&prefix={}/">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(
            bucket, region, bias_s3_prefix
        )
    )
)

In [ ]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(
    processing_job_name=balanced_bias_processing_job_name, sagemaker_session=sess
)

### _このセルの実行には5〜10分程度かかります。_

In [ ]:
%%time

running_processor.wait(logs=False)

### バランス化したデータセットのバイアスレポートを分析

なお、クラス不均衡のメトリクスは、ターゲットラベルに対してすべての商品カテゴリーで同じ値になっています。

S3から生成されたバイアスレポートをダウンロード

In [ ]:
!aws s3 ls $bias_report_balanced_output_path/

In [ ]:
!aws s3 cp --recursive $bias_report_balanced_output_path ./generated_bias_report/data_balanced/

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="./generated_bias_report/data_balanced/report.html">Balanced Bias Report</a></b>'
    )
)

# リソースを解放

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>